In [1]:
import numpy as np
import pandas as pd
import requests 

from sklearn.neighbors import KNeighborsRegressor

#### These are the largest CBSA's in America, and they are the ones we will use for this analysis.

In [2]:
cbsa_dict = {
    35620 : "new_york", 31080 : "los_angeles", 16980 : "chicago", 19100 : "dallas-fort_worth", 26420 : "houston", 
    37980 : "philadelphia", 47900 : "washington", 33100 : "miami", 12060 : "atlanta", 14460 : "boston", 
    41860 : "san_francisco", 40140 : "riverside", 38060 : "phoenix", 19820 : "detroit", 42660 : "seattle", 
    33460 : "minneapolis", 41740 : "san_diego", 45300 : "tampa", 41180 : "st_louis", 19740 : "denver"
}

cbsa = list(cbsa_dict.values())

#### And this is a useful look-up table of state FIPS codes.

In [3]:
resp = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME&for=state:*")
fips = {int(v[1]) : v[0] for v in resp.json()[1:]}

### API-Based ACS

You can view the data on broadband Internet subscriptions in cities or metro areas through these two API calls:

* https://api.census.gov/data/2019/acs/acs1/profile/?get=NAME,DP02_0087E,DP02_0153PE&for=place
* https://api.census.gov/data/2019/acs/acs1/profile/?get=NAME,DP02_0087E,DP02_0153PE&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area

The API's reference documentation is [here](https://www.census.gov/data/developers/data-sets/acs-1year.html).

First create a function to grab and format these data:

In [4]:
def get_acs_by_geog(geog, include_hh = False):

    api_base = "https://api.census.gov/data/2019/acs/acs1/profile?"
    api_vars = "get=NAME,DP02_0087E,DP02_0153E,DP02_0153PE,DP02_0151E&for={}"
    api_call = api_base + api_vars
    
    resp = requests.get(api_call.format(geog))

    j = resp.json()
    acs_rates = pd.DataFrame(columns = j[0], data = j[1:])

    acs_rates.rename(columns = {geog : "geoid"}, inplace = True)
    acs_rates.rename(columns = {"NAME" : geog, 
                                "DP02_0087E"  : "acs_population",
                                "DP02_0153PE" : "acs_broadband",
                                "DP02_0153E"  : "acs_nhh_broadband",
                                "DP02_0151E"  : "acs_nhh"},
                     inplace = True)
    
    acs_rates.query("~acs_broadband.isna()", engine = "python", inplace = True)

    acs_rates["geoid"] = acs_rates["geoid"].astype(int)
    acs_rates["acs_broadband"] = acs_rates["acs_broadband"].astype(float) / 100.
    acs_rates["acs_population"] = acs_rates["acs_population"].astype(int)
    acs_rates["acs_nhh"] = acs_rates["acs_nhh"].astype(int)
    acs_rates["acs_nhh_broadband"] = acs_rates["acs_nhh_broadband"].astype(float)
    
    acs_rates.set_index(geog, inplace = True)
    
    if geog == "place":
        acs_rates.index = acs_rates.index.str.replace(" city.*", "", regex = True)
    if geog == "metropolitan statistical area/micropolitan statistical area":
        # acs_rates.index = acs_rates.index.str.replace("-.*", "", regex = True)
        acs_rates.index = acs_rates.geoid.replace(cbsa_dict).str.replace("_", " ").str.title()
        acs_rates.index.name = "CBSA"

    acs_rates = acs_rates.sort_values("acs_population", ascending = False).head(20).copy()
    acs_rates.sort_values("acs_broadband", ascending = False, inplace = True)

    if include_hh: return acs_rates
    
    return acs_rates[["acs_population", "acs_broadband"]]

### Now use that function to get place, metro, or state-level estimates.

In [5]:
api_place_broadband = get_acs_by_geog("place")
api_cbsa_broadband  = get_acs_by_geog("metropolitan statistical area/micropolitan statistical area")
api_state_broadband = get_acs_by_geog("state", include_hh = True)

Consider the differences between place and metro estimates.

Note that these are not always available, since I have top-twentied this, and the overlap is imperfect.

In [6]:
(api_cbsa_broadband.acs_broadband - api_place_broadband.acs_broadband).dropna()

Chicago          0.041
Denver           0.010
Houston          0.041
Los Angeles      0.016
New York         0.027
Philadelphia     0.048
Phoenix          0.041
San Diego       -0.001
San Francisco    0.021
Seattle         -0.001
Washington       0.050
Name: acs_broadband, dtype: float64

### Now get individual-level data from IPUMS (Integrated Public Use Microdata Samples)

This sample needs to be constructed with your own account here: https://usa.ipums.org/usa/index.shtml

Load in the data.

In [7]:
ipums = pd.read_csv("ipums_2019.csv.gz", 
                    usecols = ["YEAR", "SAMPLE", "PERWT", "HHWT", "PERNUM",
                               "STATEFIP", "COUNTYFIP", "METRO", "MET2013", "CITY",
                               "PUMA", "GQ", "HHINCOME", "CILAPTOP", "CISMRTPHN", "CITABLET", #"CIHAND", 
                               "CINETHH", "CIDATAPLN", "CIHISPEED", "CISAT", "CIDIAL", "CIOTHSVC", 
                               "SEX", "AGE", "RACE", "HISPAN", "RACBLK", "RACWHT", "EDUC", 
                               "INCTOT", "FTOTINC", "INCWAGE", "INCEARN", "POVERTY"])

The codes can be hard to read, so let's clean them up a little.

For reference, see the pages like this: https://usa.ipums.org/usa-action/variables/CINETHH

In [8]:
# Create state names, and rename geography columns for convenience.
ipums["state"] = ipums.STATEFIP.replace(fips)
# ipums.rename(columns = {"STATEFIP" : "STATE", "COUNTYFIP" : "COUNTY"}, inplace = True)

# Educational attainement is bachelor's or higher
ipums["BA"] = ipums["EDUC"] >= 10

# Group quarters
ipums["isGQ"] = ipums.GQ.isin([3, 4, 5])

# Categorical variables made comprehensible.
ipums["sex"]  = np.where(ipums.SEX == 1,      "Male", "Female")
ipums["blk"]  = np.where(ipums.RACBLK == 2,   "Black", "Non-Black")
ipums["hisp"] = np.where(ipums.HISPAN > 0,    "Hispanic", "Non-Hispanic")
ipums["wht"]  = np.where(ipums.RACWHT == 2,   "White", "Non-White")
ipums["pov"]  = np.where(ipums.POVERTY < 100, "Below Poverty Line", "Above Poverty Line")

ipums["any_access"] = np.where(ipums.CIHISPEED == 0, np.nan,
                               ipums.CINETHH.isin([1, 2]).astype(int))
ipums["broadband"] = np.where( ipums.CIHISPEED == 0, np.nan,
                              (ipums.CIHISPEED == 10).astype(int))

# 1 is yes, 2 is no
ipums["data_plan"] = (ipums.CIDATAPLN == 1).astype(int)
ipums["satellite"] = (ipums.CISAT == 1).astype(int)
ipums["laptop"] = ipums.CILAPTOP == 1
ipums["smartphone"] = ipums.CISMRTPHN == 1

# Economists love logarithms.
ipums["logFInc"]  = np.log(ipums["FTOTINC"]) .replace({-np.inf : np.nan})
ipums["logInc"]   = np.log(ipums["INCTOT"])  .replace({-np.inf : np.nan})
ipums["logHHInc"] = np.log(ipums["HHINCOME"]).replace({-np.inf : np.nan})

/home/jsaxon/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/jsaxon/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Define functions for averages weighted by household or person.

In [9]:
def h_weighted_mean(grp): return grp._get_numeric_data().multiply(grp['HHWT'], axis=0).sum()/grp['HHWT'].sum()
def p_weighted_mean(grp): return grp._get_numeric_data().multiply(grp['PERWT'], axis=0).sum()/grp['PERWT'].sum()

### Compare levels by state and household with our results from the API.

Compare counts for various definitions of brooadband subscriptions or accesss, with a denominator of non-GQ or all.

The broadband subscription variable, [`CIHISPEED`](https://usa.ipums.org/usa-action/variables/CIHISPEED), did not reproduce state-level API results whereas the more-general "access" variable, [`CINETHH`](https://usa.ipums.org/usa-action/variables/CINETHH#codes_section), did if requiring group quarters.

In [10]:
ipums_hh = ipums.query("PERNUM == 1").copy()

ipums_hh["HHWT_noGQ"] = (~ipums_hh["isGQ"]) * ipums_hh["HHWT"]
ipums_hh["any_HHWT"] = ipums_hh.any_access * ipums_hh["HHWT"]
ipums_hh["bband_HHWT"] = ipums_hh.broadband * ipums_hh["HHWT"]
ipums_hh["any_HHWT_noGQ"] = ipums_hh.any_access * ipums_hh["HHWT"] * (~ipums_hh["isGQ"])
ipums_hh["bband_HHWT_noGQ"] = ipums_hh.broadband * ipums_hh["HHWT"] * (~ipums_hh["isGQ"])

ipums_hh.groupby("state")[['HHWT', 'HHWT_noGQ', 
                           'any_HHWT', 'bband_HHWT',
                           'any_HHWT_noGQ', 'bband_HHWT_noGQ']].sum()\
           .sort_values(by = "HHWT", ascending = False).head(10)

,HHWT,HHWT_noGQ,any_HHWT,bband_HHWT,any_HHWT_noGQ,bband_HHWT_noGQ
state,,,,,,
California,13984408.0,13156759.0,11855139.0,9988197.0,11854103.0,9987161.0
Texas,10585803.0,9985032.0,8624148.0,6748549.0,8624070.0,6748549.0
Florida,8335808.0,7905719.0,6885495.0,5755975.0,6885381.0,5755861.0
New York,8015430.0,7446483.0,6441855.0,5461878.0,6441521.0,5461544.0
Pennsylvania,5540985.0,5118937.0,4386417.0,3701208.0,4386114.0,3700905.0
Illinois,5162821.0,4865842.0,4204328.0,3424488.0,4204266.0,3424426.0
Ohio,5047619.0,4730333.0,4048736.0,3337615.0,4048736.0,3337615.0
North Carolina,4327238.0,4046248.0,3456050.0,2861906.0,3455991.0,2861847.0
Michigan,4193389.0,3969808.0,3421414.0,2707601.0,3421332.0,2707601.0


In [11]:
api_state_broadband.sort_values("acs_population", ascending = False).head(10)[["acs_nhh", "acs_nhh_broadband"]]

,acs_nhh,acs_nhh_broadband
state,,
California,13157873,11816535.0
Texas,9985126,8613544.0
Florida,7905832,6863866.0
New York,7446812,6422841.0
Pennsylvania,5119249,4379848.0
Illinois,4866006,4183116.0
Ohio,4730340,4038660.0
Georgia,3852714,3276312.0
North Carolina,4046348,3449589.0


So their denominator seems to be non-GQ ,and the numerator is *any* rather than truly broadband.

Using this construction, let's check `acs_broadband` versus `any_access`.

In [12]:
ipums_hh = ipums.query("(PERNUM == 1) & (~isGQ)").copy()

ipums_access = ipums_hh.groupby("state").apply(h_weighted_mean)\
                 [["any_access", "broadband", "data_plan", "laptop", "smartphone"]]

api_state_broadband.join(ipums_access)[["acs_broadband", "any_access", "broadband", "data_plan"]].corr()

,acs_broadband,any_access,broadband,data_plan
acs_broadband,1.000000,0.997955,0.891729,0.954213
any_access,0.997955,1.000000,0.881935,0.950716
broadband,0.891729,0.881935,1.000000,0.799478
data_plan,0.954213,0.950716,0.799478,1.000000


So just to be clear, there are additional, implicit ways that the nominal statistics are not capturing the extent of the digital divide: group quarters is hardly even by race etc.

In [13]:
ipums.groupby("blk").isGQ.mean()

blk
Black        0.102437
Non-Black    0.040493
Name: isGQ, dtype: float64

### Digital Divides within CBSAs

We will now do this calculation for *people* instead of households (not requiring `PERNUM == 1`).

That also means `p_weighted_mean` instead of `h_weighted_mean`.

But we will still drop group quarters: otherwise, the variable is not defined.

We are using CBSAs instead of cities, because cities are not available in the IPUMS data.

In [14]:
ipums_cbsa = ipums.query("(MET2013 in @cbsa_dict) & ~isGQ").copy()
ipums_cbsa["cbsa"] = ipums_cbsa.MET2013.replace(cbsa_dict)

In [15]:
agg = ipums_cbsa.groupby("cbsa").apply(p_weighted_mean)\
                [["any_access", "broadband", "data_plan", "laptop", "smartphone"]].round(3)
    
agg = agg.reindex(cbsa)
agg.index = agg.index.str.replace("_", " ").str.title()
agg.sort_values("any_access", ascending = False)

,any_access,broadband,data_plan,laptop,smartphone
cbsa,,,,,
San Diego,0.956,0.847,0.903,0.891,0.949
Seattle,0.955,0.859,0.907,0.902,0.949
Denver,0.949,0.838,0.894,0.888,0.942
San Francisco,0.948,0.850,0.912,0.891,0.949
Washington,0.940,0.845,0.885,0.895,0.946
Boston,0.938,0.852,0.876,0.876,0.918
Minneapolis,0.934,0.816,0.865,0.885,0.934
Los Angeles,0.923,0.784,0.851,0.848,0.943
Riverside,0.923,0.789,0.814,0.843,0.940


### Digital Divides

In [16]:
divide = []
for v, label, low, high in [["sex",  "Sex",     "Female", "Male"],
                            ["blk",  "Race",    "Black", "Non-Black"],
                            ["hisp", "Ethn.",   "Hispanic", "Non-Hispanic"],
                            ["pov",  "Poverty", "Below Poverty Line", "Above Poverty Line"]]:
    
    X = ipums_cbsa.groupby(["cbsa", v]).apply(p_weighted_mean).broadband.unstack()
    X["Δ" + label] = X[high] - X[low]
    divide.append(X)


divide = pd.concat(divide, axis = 1).round(3)
# divide = divide.filter(regex = "Δ", axis = 1)

divide.index = divide.index.str.replace("_", " ").str.title()
divide.sort_values(by = "ΔRace", ascending = False, inplace = True)

In [17]:
ipums_inc = ipums_cbsa[["cbsa", "broadband", "blk", "hisp", "sex", 
                        "logHHInc", "BA", "AGE", "PERWT"]].dropna()
ipums_inc["BA1000"] = ipums_inc["BA"] * 1000
ipums_inc["age_dec"] = (ipums_inc["AGE"] // 10).astype(int) * 1000

In [18]:
def knn_divide(v, label, A, B, exog = ["logHHInc", "age_dec"]):

    knn_divide = {}
    for c in cbsa:

        dfA = ipums_inc.query(f"(cbsa == '{c}') & ({v} == '{A}')").reset_index(drop = True)
        dfB = ipums_inc.query(f"(cbsa == '{c}') & ({v} == '{B}')").reset_index(drop = True)

        knn = KNeighborsRegressor(n_neighbors = 10, weights = "distance")

        knn.fit(dfA[exog], dfA.broadband)

        dfB["broadband_A_pred"] = knn.predict(dfB[exog])
        dfB["broadband_diffAB_pred"] = dfB["broadband_A_pred"] - dfB["broadband"]
        knn_divide[c] = (dfB["broadband_diffAB_pred"] * dfB["PERWT"]).sum() / dfB["PERWT"].sum()
    
    knn_divide =  pd.Series(knn_divide)
    knn_divide.index = knn_divide.index.str.replace("_", " ").str.title()
    
    knn_divide.name = "Δk " + label

    return knn_divide

In [19]:
exog = ["logHHInc", "age_dec"]

race_divide = knn_divide("blk",  "Race",  "Non-Black",    "Black",    exog = exog)
sex_divide  = knn_divide("sex",  "Sex",   "Male",         "Female",   exog = exog)
ethn_divide = knn_divide("hisp", "Ethn.", "Non-Hispanic", "Hispanic", exog = exog)

knn_divides = pd.concat([race_divide, sex_divide, ethn_divide], axis = 1).round(3)

In [20]:
all_divide = divide.join(knn_divides)

all_divide = all_divide[['Female', 'Male', 'ΔSex', 'Δk Sex',
                         'Black', 'Non-Black', 'ΔRace', 'Δk Race', 
                         'Hispanic', 'Non-Hispanic', 'ΔEthn.', 'Δk Ethn.',
                         'Above Poverty Line', 'Below Poverty Line', 'ΔPoverty']].copy()

In [21]:
all_divide.filter(regex = "Δ", axis = 1).corr().round(3)\
           .style.background_gradient(axis=None, vmin=0, vmax=1, cmap="viridis").format("{:.3f}")

,ΔSex,Δk Sex,ΔRace,Δk Race,ΔEthn.,Δk Ethn.,ΔPoverty
ΔSex,1.000,0.259,0.465,0.432,-0.451,-0.306,-0.017
Δk Sex,0.259,1.000,-0.035,0.632,0.175,0.485,0.107
ΔRace,0.465,-0.035,1.000,0.642,-0.429,-0.341,0.490
Δk Race,0.432,0.632,0.642,1.000,-0.242,0.037,0.362
ΔEthn.,-0.451,0.175,-0.429,-0.242,1.000,0.658,-0.106
Δk Ethn.,-0.306,0.485,-0.341,0.037,0.658,1.000,-0.074
ΔPoverty,-0.017,0.107,0.490,0.362,-0.106,-0.074,1.000


In [22]:
all_divide["one"] = 1

all_divide.sort_values("ΔRace", ascending = False, inplace = True)
all_divide["N ΔRace"] = all_divide["one"].cumsum()

all_divide.sort_values("Δk Race", ascending = False, inplace = True)
all_divide["N Δk Race"] = all_divide["one"].cumsum()

In [23]:
all_divide.sort_values("Δk Race", ascending = False, inplace = True)

all_divide[["N Δk Race", "Δk Race", "N ΔRace", "ΔRace", 
            "Δk Ethn.", "ΔEthn.", "Δk Sex", "ΔSex", "ΔPoverty"]]

,N Δk Race,Δk Race,N ΔRace,ΔRace,Δk Ethn.,ΔEthn.,Δk Sex,ΔSex,ΔPoverty
cbsa,,,,,,,,,
Detroit,1,0.069,1,0.162,0.026,0.031,0.009,0.021,0.220
Tampa,2,0.063,6,0.094,0.064,0.052,0.015,0.018,0.217
Chicago,3,0.056,4,0.117,0.053,0.081,0.006,0.008,0.208
Phoenix,4,0.056,14,0.047,0.096,0.151,0.013,0.002,0.203
Miami,5,0.054,7,0.094,0.021,0.028,0.009,0.010,0.194
Philadelphia,6,0.047,3,0.120,0.030,0.082,0.004,0.016,0.188
New York,7,0.032,10,0.074,0.033,0.060,0.004,0.014,0.220
San Francisco,8,0.032,5,0.103,0.055,0.102,0.005,0.016,0.222
Boston,9,0.032,8,0.088,0.075,0.126,0.004,0.006,0.173


In [24]:
api_rate = api_cbsa_broadband.sort_values(by = "acs_population", ascending = False)\
                  [["acs_broadband"]].rename(columns = {"acs_broadband" : "Access"}).copy()

all_vars = api_rate.join(all_divide[["Δk Race", "ΔRace", "Δk Ethn.", "ΔEthn.", "Δk Sex", "ΔSex"]])

In [25]:
all_vars * 100

,Access,Δk Race,ΔRace,Δk Ethn.,ΔEthn.,Δk Sex,ΔSex
CBSA,,,,,,,
New York,87.8,3.2,7.4,3.3,6.0,0.4,1.4
Los Angeles,89.3,0.4,4.1,5.8,9.6,0.6,0.8
Chicago,87.7,5.6,11.7,5.3,8.1,0.6,0.8
Dallas-Fort Worth,89.5,0.4,6.3,8.2,13.4,0.1,0.6
Houston,89.1,-1.8,2.5,5.9,12.1,-0.4,-0.1
Washington,92.3,1.1,5.4,3.0,8.6,-0.5,0.3
Miami,84.7,5.4,9.4,2.1,2.8,0.9,1.0
Philadelphia,89.0,4.7,12.0,3.0,8.2,0.4,1.6
Atlanta,89.6,2.5,6.8,-0.2,9.7,0.6,0.6


In [26]:
print((all_vars.round(3) * 100).to_markdown())

| CBSA              |   Access |   Δk Race |   ΔRace |   Δk Ethn. |   ΔEthn. |   Δk Sex |   ΔSex |
|:------------------|---------:|----------:|--------:|-----------:|---------:|---------:|-------:|
| New York          |     87.8 |       3.2 |     7.4 |        3.3 |      6   |      0.4 |    1.4 |
| Los Angeles       |     89.3 |       0.4 |     4.1 |        5.8 |      9.6 |      0.6 |    0.8 |
| Chicago           |     87.7 |       5.6 |    11.7 |        5.3 |      8.1 |      0.6 |    0.8 |
| Dallas-Fort Worth |     89.5 |       0.4 |     6.3 |        8.2 |     13.4 |      0.1 |    0.6 |
| Houston           |     89.1 |      -1.8 |     2.5 |        5.9 |     12.1 |     -0.4 |   -0.1 |
| Washington        |     92.3 |       1.1 |     5.4 |        3   |      8.6 |     -0.5 |    0.3 |
| Miami             |     84.7 |       5.4 |     9.4 |        2.1 |      2.8 |      0.9 |    1   |
| Philadelphia      |     89   |       4.7 |    12   |        3   |      8.2 |      0.4 |    1.6 |
| Atlanta 

### Cities??

Starting from Houston (and then San Antonio, San Diego, Dallas, San Jose, etc.), cities are irregularly available in the IPUMS data.  See [variable codes](https://usa.ipums.org/usa-action/variables/CITY#codes_section).

In [27]:
cities = ["new_york", "los_angeles", "chicago", "houston", "phoenix",
          "philadelphia", "san_antonio", "san_diego", "dallas",
          "san_jose", "austin", "jacksonville", "san_francisco", "columbus",
          "fort_worth", "indianapolis", "charlotte", "seattle", "denver", "washington"]

city_dict = {4610 : "new_york", 3730 : "los_angeles",  1190 : "chicago",     2890 : "houston", 5350 : "phoenix",  
             5330 : "philadelphia", 6230 : "san_antonio", 6270 : "san_diego", 1590 : "dallas", 6310 : "san_jose",
              490 : "austin", 3110 : "jacksonville", 6290 : "san_francisco", 1450 : "columbus", 2350 : "fort_worth",
             2990 : "indianapolis", 1090 : "charlotte", 6430 : "seattle", 1710 : "denver", 7230 : "washington"}